In [5]:
# Libraries 
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import plotly.express as px
import ta
# import talib
from ta import add_all_ta_features
import pytz

#Pandas display Options
pd.set_option('display.max_columns', None)
# pd.set_option("max_rows", 300)

# connect to MetaTrader5 as mt5
if mt5.initialize():
    print('intialized successfully!')
    
else:
    print('initialization failed!')
    mt5.quit()

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

intialized successfully!


In [4]:
#GET DATA
def get_data(symbol, timeframe):
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        #mt5.shutdown()

    df = mt5.copy_rates_from_pos(symbol, timeframe, 0, 2000)
    
    # create DataFrame out of the obtained data
    df = pd.DataFrame(df)
    # convert time in seconds into the datetime format
    df['time']=pd.to_datetime(df['time'], unit='s')
    df.index = df.time.values
    df = df.drop(["time", "spread", "real_volume"], axis = 1)
    df.columns = ["Open", "High", "Low", "Close", "Volume"]
    
    df = df.dropna()
    return df

In [ ]:
timef = mt5.TIMEFRAME_D1 #mt5.TIMEFRAME_M5
df = get_data('EURCAD', timef)
df

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import ta
# import talib
from ta import add_all_ta_features
import pytz
from tvDatafeed import TvDatafeed, Interval

timeframe_tv = {
    'MN1': Interval.in_monthly,
    'W1': Interval.in_weekly,
    'D1': Interval.in_daily,
    'H4': Interval.in_4_hour,
    'H1': Interval.in_1_hour,
    'M15': Interval.in_15_minute,
    'M30': Interval.in_30_minute,
    'M5': Interval.in_5_minute,
    'M1': Interval.in_1_minute,
    # None: Interval.in_daily,
}

def get_data(symbol, timeframe, exchange):

    tv = TvDatafeed()
    #   exchange='NASDAQ'
    df = tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe_tv[timeframe],
                        n_bars=6000)  # Interval.in_1_hour
    # create DataFrame out of the obtained data
    df = pd.DataFrame(df).reset_index()
    # # convert time in seconds into the datetime format
    df['time'] = pd.to_datetime(df.index, unit='s')
    #df['time'] = [i + datetime.timedelta(days=1) for i in df['time']]
    #df.index = df.time.values
    df = df.drop(["time", "symbol"], axis=1)  # "open", "high", "low"
    df = df.rename(columns={"open": "Open",
                          "close": "Close",
                          "high": "High",
                          "low": "Low",
                           "volume": "Volume"})
    df.dropna(inplace=True)

    return df

def signal(df):
#     buy_conditions = 
#     sell_conditions = df['trend_ema_fast'] < df['trend_ema_slow'] and df['momentum_rsi'] > 60.0
    
    df['signal'] = np.where((df['trend_ema_fast'] > df['trend_ema_slow']) & (df['momentum_rsi'] > 60.0), 'Buy', 'Neutral')
    df['signal'] = np.where((df['trend_ema_fast'] < df['trend_ema_slow']) & (df['momentum_rsi'] > 60.0), 'Sell', df['signal'])
    
    return df
    

In [7]:
symbol = 'EURUSD'
timeframe = 'D1'
# timef = mt5.TIMEFRAME_D1
EXCHANGE = 'OANDA'
df = get_data(symbol, timeframe, EXCHANGE)
# df = get_data(symbol, timef)
df = add_all_ta_features(df, open='Open', high='High', low='Low', close='Close', volume='Volume')
df

you are using nologin method, data you access may be limited


,datetime,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2002-05-06 22:00:00,0.91535,0.91535,0.91535,0.91535,1.0,0.000000e+00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.915350,0.915350,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.915350,NaN,0.000000,0.000000,0.000000,NaN,1.226302,1.22486,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2002-05-07 22:00:00,0.91535,0.91535,0.90435,0.90435,1.0,-1.000000e+00,0.0,NaN,NaN,-6050.000000,NaN,-0.012017,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.917183,0.906183,NaN,-0.166667,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.909850,NaN,0.000000,0.000000,0.000000,NaN,1.226302,1.22486,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.201726,-1.209005,-1.201726
2,2002-05-08 22:00:00,0.90435,0.90935,0.90435,0.90935,1.0,0.000000e+00,1.0,NaN,NaN,-1500.000000,NaN,-0.006488,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.915683,0.905017,NaN,0.406250,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.909850,NaN,0.000000,0.000000,0.000000,NaN,1.226302,1.22486,NaN,NaN,NaN,NaN,0.915350,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.552883,0.551360,-0.655487
3,2002-05-09 22:00:00,0.90935,0.91400,0.90935,0.91400,1.0,1.000000e+00,2.0,NaN,NaN,2243.625000,NaN,-0.001375,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.916038,0.905712,NaN,0.802663,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.909850,NaN,0.000000,0.000000,0.000000,NaN,1.226302,1.22486,NaN,NaN,NaN,NaN,0.915350,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.511354,0.510051,-0.147485
4,2002-05-12 22:00:00,0.91400,0.91400,0.91155,0.91155,1.0,0.000000e+00,1.0,NaN,NaN,269.500000,NaN,-0.004055,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.915793,0.906553,NaN,0.540765,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.909850,NaN,0.000000,0.000000,0.000000,NaN,1.226302,1.22486,NaN,NaN,NaN,NaN,0.915130,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.268053,-0.268412,-0.415142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [8]:
df.columns

Index(['datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       '

In [33]:
from datetime import date

date.today()

datetime.date(2024, 1, 23)

In [37]:
data_list[0]['datetime'].iloc[-1].date()

datetime.date(2024, 1, 22)

In [57]:
# class Position contain data about trades opened/closed during the backtest 
class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp, com):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.com = com
        self.close_datetime = None
        self.close_price = None
        self.profit = 0
        self.net_profit = self.profit + self.com
        self.status = 'open'
        self.closedonce = False
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.returns = round(((self.close_price - self.open_price) / self.open_price), 5)
#         self.returns = np.log(self.close_price/self.open_price)
        self.status = 'closed'
        
    def _asdict(self):

        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
#             'tp_half': self.tp_half,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'returns': self.returns,
            'profit': self.profit,
            'com' : self.com,
            'Net Profit' : self.profit + self.com,
            'status': self.status,
#             'closed_once' : self.closedonce
        }
        

# class Strategy defines trading logic and evaluates the backtest based on opened/closed positions
class Strategy:
    def __init__(self, df, starting_balance, sl_pips, tp_pips):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df
        self.sl_pips = sl_pips
        self.tp_pips = tp_pips
        
    
    # return backtest result
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['Net Profit'].cumsum() + self.starting_balance
        return df
    
    # add Position class to list
    def add_position(self, position):
        self.positions.append(position)
        return True
    
    # close half positions when trade goes 10pips in profit
    def close_tp_sl(self, data):
        for pos in self.positions:
            if pos.status == 'open':

                if (pos.tp <= data.Close and pos.order_type == 'buy'):
                    pos.close_position(data.datetime, pos.tp)
                    
                elif (pos.tp >= data.Close and pos.order_type == 'sell'):
                    pos.close_position(data.datetime, pos.tp)
                                        
                elif (pos.sl >= data.Close and pos.order_type == 'buy'):
                    pos.close_position(data.datetime, pos.sl)
                elif (pos.sl <= data.Close and pos.order_type == 'sell'):
                    pos.close_position(data.datetime, pos.sl)
                    
                elif data['datetime'].date() >= date.today():
                    pos.close_position(data.datetime, data.Close)
                    
                    
            #elif pos.status == "half closed":
                #pos.modify_sl()

    # check for open positions
    def has_open_positions(self):
        for pos in self.positions:
            if pos.status == 'open':
                return True
        return False
    
    # strategy logic how positions should be opened/closed
    def logic(self, data):
        # if no position is open
        if not self.has_open_positions():
            
            # BUY
            if data['signal'] == 'Buy':

                # Position variables
                open_datetime = data['datetime']
                open_price = data['Close']
                order_type = 'buy'
                volume = 100000
                com = - 7.00
                sl = open_price - self.sl_pips/10000
                tp = open_price + self.tp_pips/10000
#                 tp_half = open_price + 0.00100

                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp, com))
                

            # SELL
            elif data['signal'] == 'Buy':

                # Position variables
                open_datetime = data['datetime']
                open_price = data['Close']
                order_type = 'sell'
                volume = 100000 #100xau, 1000ypy, 100000REG
                com = - 7.00
                sl = open_price + self.sl_pips/10000 #1.00XAU #0.100JPY, 0.00100REG #10.00
                tp = open_price - self.tp_pips/10000
#                 tp_half = open_price - 0.00100

                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp, com))
            
            
        
# logic
    def run(self):
        # data represents a moment in time while iterating through the backtest
        for i, data in self.data.iterrows():
            # close positions when stop loss or take profit is reached
            self.close_tp_sl(data)
            
            # strategy logic
            self.logic(data)
        
        return self.get_positions_df()

In [8]:
df_backtest

,index


In [58]:
timeframe = 'D1'
EXCHANGE = 'OANDA'

pairs = ["GBPCAD"] #, "AUDUSD", "USDCAD", "USDCHF", "AUDCAD", "CADCHF", "NZDUSD", "EURCAD", "AUDCHF", "GBPUSD",
#          "GBPCAD", "GBPNZD", "AUDNZD", "EURGBP", "EURNZD", "GBPCHF", "EURCHF", "EURAUD", "NZDCAD", "NZDCHF", "GBPAUD",
#         "GBPJPY", "CADJPY", "EURJPY", "AUDJPY", "NZDJPY","USDJPY","CHFJPY"
#         ]
pairs_jpy = ["GBPJPY", "CADJPY", "EURJPY", "AUDJPY", "NZDJPY","USDJPY","CHFJPY"]



data_list = []
for inst in pairs: 
    df_backtest = get_data(inst, timeframe, EXCHANGE)
    df_backtest = add_all_ta_features(df_backtest, open='Open', high='High', low='Low', close='Close', volume='Volume')
    df_backtest = signal(df_backtest)
    df_backtest['Ticker'] = inst
    df_backtest['timeframe'] = timeframe
#     df_backtest.dropna(inplace = True)
#     df_backtest = df_backtest.reset_index()
#     df_backtest = df_backtest.rename(columns={"index":"time", "Close":"close"})
    data_list.append(df_backtest)

In [59]:
data = pd.concat(data_list)
# data['timeframe'] = 'D1'
data

,datetime,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,signal,Ticker,timeframe
0,2004-05-31 22:00:00,2.51700,2.51700,2.51700,2.51700,2.0,0.000000e+00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.517000,2.517000,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.517000,NaN,0.000000,0.000000,0.000000,NaN,1.801704,1.807572,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Neutral,GBPCAD,D1
1,2004-06-01 22:00:00,2.51700,2.51700,2.49790,2.49790,2.0,-2.000000e+00,0.0,NaN,NaN,-9120.250000,NaN,-0.015177,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.520183,2.501083,NaN,-0.166667,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.507450,NaN,0.000000,0.000000,0.000000,NaN,1.801704,1.807572,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.758840,-0.761734,-0.758840,Neutral,GBPCAD,D1
2,2004-06-02 22:00:00,2.49790,2.50835,2.49790,2.50835,2.0,0.000000e+00,2.0,NaN,NaN,-2259.812500,NaN,-0.006810,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.518561,2.498861,NaN,0.481669,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.507450,NaN,0.000000,0.000000,0.000000,NaN,1.801704,1.807572,NaN,NaN,NaN,NaN,2.517000,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.418351,0.417479,-0.343663,Neutral,GBPCAD,D1
3,2004-06-03 22:00:00,2.50835,2.50835,2.48655,2.48655,2.0,-2.000000e+00,0.0,NaN,NaN,-6185.750000,NaN,-0.024192,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.517825,2.492150,NaN,-0.218111,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.501775,NaN,0.000000,0.000000,0.000000,NaN,1.801704,1.807572,NaN,NaN,NaN,NaN,2.517000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.869097,-0.872896,-1.209774,Neutral,GBPCAD,D1
4,2004-06-06 22:00:00,2.48655,2.48655,2.48000,2.48000,2.0,-4.000000e+00,-2.0,NaN,NaN,-4642.312500,NaN,-0.029460,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2.512007,2.488847,NaN,-0.381980,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.498500,NaN,0.000000,0.000000,0.000000,NaN,1.801704,1.807572,NaN,NaN,NaN,NaN,2.515782,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.263417,-0.263765,-1.470004,Neutral,GBPCAD,D1
...,...,...,...,...,...,...,...,...,...,...,...,

In [28]:
data.to_csv(f'{timeframe}_data_FX.csv')

In [60]:
strategy_name = "demo"
starting_balance = 100000
sl_pips = 10
tp_pips = 50
#batch
results_0 = []
for symbol, data in zip (pairs, data_list):
    strategy = Strategy(data, starting_balance, sl_pips, tp_pips)
    result = strategy.run()
    result["symbol"] = symbol
    results_0.append(result)
    

In [61]:
results = pd.concat(results_0)
results

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,returns,profit,com,Net Profit,status,pnl,symbol
0,2004-11-30 23:00:00,2.28730,buy,100000,2.28630,2.29230,2004-12-01 23:00:00,2.29230,0.00219,500.0,-7.0,493.0,closed,100493.0,GBPCAD
1,2004-12-01 23:00:00,2.29810,buy,100000,2.29710,2.30310,2004-12-02 23:00:00,2.30310,0.00218,500.0,-7.0,493.0,closed,100986.0,GBPCAD
2,2004-12-02 23:00:00,2.31615,buy,100000,2.31515,2.32115,2004-12-05 23:00:00,2.32115,0.00216,500.0,-7.0,493.0,closed,101479.0,GBPCAD
3,2004-12-05 23:00:00,2.32830,buy,100000,2.32730,2.33330,2004-12-06 23:00:00,2.33330,0.00215,500.0,-7.0,493.0,closed,101972.0,GBPCAD
4,2004-12-06 23:00:00,2.35030,buy,100000,2.34930,2.35530,2004-12-09 23:00:00,2.34930,-0.00043,-100.0,-7.0,-107.0,closed,101865.0,GBPCAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,2023-11-19 23:00:00,1.71622,buy,100000,1.71522,1.72122,2023-11-21 23:00:00,1.71522,-0.00058,-100.0,-7.0,-107.0,closed,183771.0,GBPCAD
647,2023-11-21 23:00:00,1.71014,buy,100000,1.70914,1.71514,2023-11-22 23:00:00,1.71514,0.00292,500.0,-7.0,493.0,closed,184264.0,GBPCAD
648,2023-11-22 23:00:00,1.71689,buy,100000,1.71589,1.72189,2023-11-27 23:00:00,1.72189,0.00291,500.0,-7.0,493.0,closed,184757.0,GBPCAD
649,2023-11-27 23:00:00,1.72338,buy,100000,1.72238,1.72838,2023-11-29 23:00:00,1.72238,-0.00058,-100.0,-7.0,-107.0,closed,184650.0,GBPCAD


In [31]:
results.to_csv('backtest_results.csv')